In [1]:
from wrappers import graph_from_point_wrapper
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import display, Image
import time
import googlemaps
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
key = config['default']['googleApiKey']
gmaps = googlemaps.Client(key=key)
coords = { 'lat': 36.005800 , 'lng': -78.897020 }

In [3]:
# Debouncing shit
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
        return debounced
    return decorator

In [4]:
def read_and_return_img():
    file = open("images/test.jpg", "rb")
    image = file.read()
    return image

In [5]:
def format_caption():
    return 'Image below: map with {radius} meter radius; {fig_size}in. figure size; {color}'.format(
        radius=radius_slider.value,
        fig_size=fig_slider.value,
        color=color_picker.value) 

In [6]:
import pprint
def geocode_for_lat_lng(address):
    try:
        result = gmaps.geocode(address)
        return result[0]['geometry']['location'] # dict with keys 'lat' & 'lng'
    except:
        return False

In [7]:
def create_and_display_img():
    graph_from_point_wrapper(
        point=(lat_text.value, lng_text.value),
        distance=radius_slider.value,
        fig_height=fig_slider.value,
        network_type='drive',
        distance_type='bbox',
        color=color_picker.value,
        save=True
    )
    image = read_and_return_img()
    img_display.value = image
@debounce(1.0)
def shared_event_handler(change):
    caption.value = format_caption()
    create_and_display_img()

@debounce(2.0)
def geocode_handler(change):
    coords = geocode_for_lat_lng(address_text.value)
    if coords:
        lat_text.value = coords['lat']
        lng_text.value = coords['lng']
    create_and_display_img()

In [8]:
# Widgets

address_text = widgets.Text(
    value='213 W Trinity Ave Durham NC',
    placeholder='Input an address',
    description='Address:'
)
lat_text = widgets.FloatText(
    value=coords['lat'],
    description='Lat:',
    disabled=True
)
lng_text = widgets.FloatText(
    value=coords['lng'],
    description='Lng:',
    disabled=True
)

radius_slider = widgets.IntSlider(
    min=500,
    max=5000,
    step=1000,
    description='Map radius:',
    value=1000
)
fig_slider = widgets.IntSlider(
    min=8,
    max=24,
    step=2,
    description='Figure size:',
    value=8
)

color_picker = widgets.ColorPicker(
    concise=False,
    description='Pick a color:',
    value='darkolivegreen',
    disabled=False
)

img_display = widgets.Image(
    value=read_and_return_img()
)

caption = widgets.Label(value=format_caption())

address_text.observe(geocode_handler, names='value')
radius_slider.observe(shared_event_handler, names='value')
fig_slider.observe(shared_event_handler, names='value')
color_picker.observe(shared_event_handler, names='value')

In [9]:
# Grid layout - https://ipywidgets.readthedocs.io/en/latest/examples/Layout%20Templates.html#Grid-layout
grid = GridspecLayout(10, 3, height='auto')
grid[1:10, 1:] = img_display
grid[0:1, 1:] = caption
grid[1:2, 0] = address_text
grid[2:3, 0] = lat_text
grid[3:4, 0] = lng_text
grid[4:5, 0] = radius_slider
grid[5:6, 0] = fig_slider
grid[6:7, 0] = color_picker
grid

GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\…